Кейс 4. Группировка данных. Работа со строками

ЗАДАНИЕ:

1)Вывести ID и суммы транзакций 10-ти персонажей, потративших наибольшее количество валюты по таблице shop_log за период с 01.08.2018 по 07.08.2018 включительно. Количество валюты считать как цену товара, умноженную на количество товара (поля itemprice и itemcount). Данные отсортировать по сумме транзакций по убыванию.

2)Вывести данные из таблицы characters по списку персонажей из предыдущего пункта, используя подзапрос. Подсказка: в подзапросе должен быть только список charid, по которому необходимо сделать выборку из таблицы characters, от сумм транзакций можно избавиться.

3)Извлечь значения параметров actionname, kill, death и assist из поля actiondata таблицы pvp_log при помощи регулярных выражений и подсчитать средние значения параметров kill, death и assist для всех actionname. Подзказка: не забывайте учитывать тип данных, с которым работаете. После извлечения числовых параметров в виде строки необходимо преобразовать их тип в числовой.

4)В одном запросе при помощи условного выражения CASE подсчитать количество строк в таблице dungeon_log, разбив данные на две группы — относящиеся к actionname ‘Нага-банк’ и остальные. То есть в результате должна получиться таблица 2х2, где первый столбец будет содержать значения “Нага-банк” и “Остальные”, а второй столбец — количество строк для этих двух групп.

In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'

from sqlalchemy import create_engine
engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')



In [3]:
import pandas as pd
import matplotlib

df = pd.read_sql(f'''
SELECT charid, sum(itemprice*itemcount) AS item_sum
FROM (
    SELECT *
    FROM gu_game_analytics_1188.shop_log
    WHERE time between '2018-08-01 00:00:00' AND '2018-08-07 23:59:59'
    ) AS t
GROUP BY charid
ORDER BY item_sum DESC
LIMIT 10
''', con=engine)
df

,charid,item_sum
0,5715311,17450.0
1,5730766,17050.0
2,4855383,16100.0
3,5089401,12350.0
4,2865146,11200.0
5,5732109,11050.0
6,2872590,11000.0
7,3875198,10700.0
8,4434267,9150.0
9,5800830,9150.0


In [4]:
df = pd.read_sql(f'''
SELECT *
FROM gu_game_analytics_1188.characters
WHERE id in (
    SELECT charid AS id
    FROM (
        SELECT charid, sum(itemprice*itemcount) AS item_sum
        FROM (
            SELECT *
            FROM gu_game_analytics_1188.shop_log
            WHERE time between '2018-08-01 00:00:00' AND '2018-08-07 23:59:59'
            ) AS t
        GROUP BY charid
        ORDER BY item_sum DESC
        LIMIT 10
    ) AS t
)
''', con=engine)
df

,id,accountid,charclass,charrace,charlevel
0,5800830,461399912,Mage,Elf,44
1,4855383,462104863,Assasin,Orc,55
2,2865146,474020503,Warrior,Elf,59
3,2872590,479690250,Mage,Human,57
4,3875198,486312386,Mage,Elf,56
5,5732109,486446920,Mage,Dwarf,49
6,5730766,490142864,Mage,Dwarf,53
7,4434267,495164907,Archer,Orc,61
8,5089401,807652165,Warrior,Orc,53
9,5715311,909228525,Mage,Dwarf,45


In [5]:
df = pd.read_sql(f'''
SELECT distinct actionname, avg(kill) as avg_kill, avg(death) as avg_death, avg(assist) as avg_assist
FROM (
    SELECT substring(actiondata, 'ActionName\":\"(\w+)') as actionname,
    cast(substring(actiondata, 'Kill\":(\d+)') as INT) as kill,
    cast(substring(actiondata, 'Death\":(\d+)') as INT) as death,
    cast(substring(actiondata, 'Assist\":(\d+)') as INT) as assist
    FROM gu_game_analytics_1068.pvp_log
    ) as t
group by actionname
''', con=engine)
df


,actionname,avg_kill,avg_death,avg_assist
0,QuickMatch,2.878404,2.885789,2.686146
1,TeamDeathMatch,1.650429,1.653565,1.265644


In [9]:
df = pd.read_sql(f'''
SELECT count(*),
    case when actionname = 'Нага-банк' then actionname
else 'Other' end as actionname_2     
FROM gu_game_analytics_1068.dungeon_log
group by actionname_2
''', con=engine)
df

,count,actionname_2
0,933250,Other
1,101940,Нага-банк
